In [2]:


from classified_cset import utils
import xarray as xr
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from scipy.interpolate import interp1d
import matplotlib as mpl
from matplotlib.colorbar import make_axes
import datetime as dt
import matplotlib.gridspec as gridspec
import glob
import pandas as pd
import numpy as np
import random
%load_ext autoreload
%autoreload 2
print('BOGIES')

BOGIES


In [3]:
cset_class_df = utils.load_class_data('cset')
class_by_gran = cset_class_df.groupby('granule')

In [3]:
# all_class_df = utils.load_class_data('all')
# sep_class_df = all_class_df[all_class_df['loc']=='SEP']
# class_by_gran = sep_class_df.groupby('granule')

In [4]:
#want scenes with at least some of each type
count_scenes_by_gran = {}
for i, df_sub in class_by_gran:
    count_dict = {i: 0 for i in range(6)}
    df_split = df_sub.groupby('cat')
    for j,k in df_split:
        count_dict[j] = len(k)
    count_scenes_by_gran[i] = count_dict 
nice_grans_2 = [g for g,f in count_scenes_by_gran.items() if np.all([f[i]>5 for i in range(6)])]

In [4]:
best_gran = 'MYD021KM.A2015203.2245'
hurricane_gran = 'MYD021KM.A2015215.0005'
good2gran = 'MYD021KM.A2015208.0000'

plot_gran = best_gran
plot_gran in class_by_gran.groups.keys()

True

In [7]:
def load_MODIS_granule(granule_name):
    mydatml2_files = glob.glob(f'/home/disk/eos4/jkcm/Data/MEASURES/MODIS_downloads/MYDATML2/MYDATML2{granule_name[8:]}*.hdf')
    myd021km_files = glob.glob(f'/home/disk/eos9/jkcm/Data/modis/MYD021KM_keep/MYD021KM{granule_name[8:]}*.hdf')
#     myd021km_files = glob.glob(f'/home/disk/eos9/jkcm/Data/modis/SEP/MYD021KM{granule_name[8:]}*.hdf')
    myd03_files = glob.glob(f'/home/disk/eos9/jkcm/Data/modis/MYD03/MYD03{granule_name[8:]}*.hdf')
#     myd03_files = glob.glob(f'/home/disk/eos9/jkcm/Data/modis/SEP/MYD03{granule_name[8:]}*.hdf')

    if not len(myd021km_files) == 1: 
        raise ValueError('no MYD02 data found for ' + granule_name)

    if not len(myd03_files) == 1: 
        raise ValueError('no MYD03 data found for ' + granule_name)
    modis_data = xr.open_dataset(myd021km_files[0])
#     atml2_data = xr.open_dataset(mydatml2_files[0])
    myd03_data = xr.open_dataset(myd03_files[0], drop_variables=['Scan Type'])
    modis_data = modis_data.rename({
                   'Band_250M:MODIS_SWATH_Type_L1B': 'Band_250M',
                   'Band_1KM_Emissive:MODIS_SWATH_Type_L1B': 'Band_1KM_Emissive',
                   'Band_1KM_RefSB:MODIS_SWATH_Type_L1B': 'Band_1KM_RefSB',
                   'Band_500M:MODIS_SWATH_Type_L1B': 'Band_500M',
                   '10*nscans:MODIS_SWATH_Type_L1B': '10*nscans',
                   '1KM_geo_dim:MODIS_SWATH_Type_L1B': '1KM_geo_dim',
                   '2*nscans:MODIS_SWATH_Type_L1B': '2*nscans',
                   'Max_EV_frames:MODIS_SWATH_Type_L1B': 'Max_EV_frames'})
    return modis_data, myd03_data

In [14]:
nice_grans = [g for g,f in count_scenes_by_gran.items() if np.all([f[i]>25 for i in [1, 5]])]
# nice_grans = random.sample(nice_grans, 10)
for gran in nice_grans:
    try:
        load_MODIS_granule(gran)
        print('worked')
    except ValueError as e:
        print(e)
        continue

no MYD02 data found for MYD021KM.A2015182.0100
no MYD02 data found for MYD021KM.A2015182.2045
no MYD02 data found for MYD021KM.A2015183.0005
no MYD02 data found for MYD021KM.A2015183.2310
no MYD02 data found for MYD021KM.A2015186.0035
no MYD02 data found for MYD021KM.A2015186.2340
no MYD02 data found for MYD021KM.A2015187.2245
no MYD02 data found for MYD021KM.A2015188.0025
no MYD02 data found for MYD021KM.A2015188.2145
no MYD02 data found for MYD021KM.A2015188.2150
no MYD02 data found for MYD021KM.A2015188.2330
no MYD02 data found for MYD021KM.A2015189.0105
no MYD02 data found for MYD021KM.A2015190.0010
no MYD02 data found for MYD021KM.A2015190.0015
no MYD02 data found for MYD021KM.A2015191.0055
no MYD02 data found for MYD021KM.A2015192.0000
no MYD02 data found for MYD021KM.A2015192.2300
no MYD02 data found for MYD021KM.A2015193.0045
no MYD02 data found for MYD021KM.A2015193.2345
no MYD02 data found for MYD021KM.A2015194.2250
no MYD02 data found for MYD021KM.A2015195.0025
no MYD02 data

In [9]:
def get_rgb_reflectance(modis_data):
    
    red_band = np.nonzero(modis_data.Band_250M.values == 1)[0][0]
    green_band = np.nonzero(modis_data.Band_500M.values == 4)[0][0]
    blue_band = np.nonzero(modis_data.Band_500M.values == 3)[0][0]
    
    red = modis_data['EV_250_Aggr1km_RefSB'].sel({'Band_250M': 1})
    green = modis_data['EV_500_Aggr1km_RefSB'].sel({'Band_500M': 4})
    blue = modis_data['EV_500_Aggr1km_RefSB'].sel({'Band_500M': 3})
    
    scales_500 = modis_data['EV_500_Aggr1km_RefSB'].reflectance_scales
    offset_500 = modis_data['EV_500_Aggr1km_RefSB'].reflectance_offsets  
    scales_250 = modis_data['EV_250_Aggr1km_RefSB'].reflectance_scales
    offset_250 = modis_data['EV_250_Aggr1km_RefSB'].reflectance_offsets
    
    red_refl = (red-offset_250[red_band])*scales_250[red_band]
    green_refl = (green-offset_500[green_band])*scales_500[green_band]
    blue_refl = (blue-offset_500[blue_band])*scales_500[blue_band]
    
    rgb = np.dstack([red_refl, green_refl, blue_refl])
    rgb[rgb<0] = 0
    rgb[rgb>1] = 1

    lin = np.array([0, 30, 60, 120, 190, 255]) / 255.0
    nonlin = np.array([0, 110, 160, 210, 240, 255]) / 255.0
    scale = interp1d(lin, nonlin, kind='quadratic')
    scaled_rgb = scale(rgb) #not necessary but prettier?
    return scaled_rgb

In [5]:
def get_divergence(date):
    ascat_file = f'/home/disk/eos9/jkcm/Data/ascat/rss/all/ascat_unified_{date.year}-{date.month:02}.nc'
    ascat_data = xr.load_dataset(ascat_file)
    ascat_data = ascat_data.sel(time=date.replace(hour=0, minute=0), method='nearest', tolerance=np.timedelta64(12, 'h'))
    ascat_data = utils.get_ascat_divergence(ascat_data)
    return(ascat_data)

def get_MERRA(date):
    MERRA_data = xr.open_dataset(r'/home/disk/eos4/jkcm/Data/CSET/MERRA/measures/MERRA_unified_subset.nc')
    MERRA_data = MERRA_data.sel(time=date, method='nearest', tolerance=np.timedelta64(3, 'h'))
#     ascat_data = ascat_data.sel(time=date.replace(hour=0, minute=0), method='nearest', tolerance=np.timedelta64(12, 'h'))
#     ascat_data = utils.get_ascat_divergence(ascat_data)
    return(MERRA_data)

def get_SEP_MERRA(date):
    MERRA_data = xr.open_dataset(
            f'/home/disk/eos4/jkcm/Data/MERRA/measures/split/MERRA_unified_subset.{date.year}-{date.month:02}.nc', lock=False)
    MERRA_data = MERRA_data.sel(time=date, method='nearest', tolerance=np.timedelta64(3, 'h'))
    return MERRA_data

def get_AMSR(date):
    amsr_data = xr.open_dataset(r'/home/disk/eos4/jkcm/Data/CSET/amsr/AMSR2_CWV_CSET_fixed.nc')        
    amsr_cwv = amsr_data.CWV.sel(time=date, method='nearest', tolerance=np.timedelta64(12, 'h'))
    return(amsr_cwv)
    
def get_precip(date):
    precip_data = xr.open_dataset(f'/home/disk/eos5/rmeast/rain_rates_89/2015/AMSR2_89GHz_pcp_est_2015_{date.dayofyear}_day.nc')
    return(precip_data)

In [10]:
# for gran in [plot_gran]:
#     df_gran = class_by_gran.get_group(gran)
#     modis_data, myd03_data = load_MODIS_granule(gran)a
#     date =  df_gran.datetime.iloc[0]
#     ascat_data = get_divergence(date)
#     ad = ascat_data.sel(longitude=slice(np.min(myd03_data.Longitude.values%360), np.max(myd03_data.Longitude.values%360)), 
#                         latitude=slice(np.min(myd03_data.Latitude.values), np.max(myd03_data.Latitude.values)),
#                         orbit_segment=0)
#     a_dates = ad.time.values + (ad.mingmt.values*60*1e3).astype('timedelta64[ms]')
#     MERRA_data = get_MERRA(date)

In [11]:
# print(f'granule: {dt.datetime.strptime(gran[10:22], "%Y%j.%H%M")}')
# print(f'classifications: {np.min(df_gran.datetime).to_datetime64()}, {np.max(df_gran.datetime).to_datetime64()}')
# print(f'date: {date}')
# print(f'ASCAT: {ad.time.values}, {np.min(a_dates)}, {np.max(a_dates)}')

In [8]:
df_gran = class_by_gran.get_group(plot_gran)
date = df_gran.datetime.iloc[0]
precip_data = get_precip(date)

In [9]:
precip_data

<xarray.Dataset>
Dimensions:     (prob_rate_rwr_max: 4, swath_width: 486, time: 57818, yr_day_utc: 3)
Dimensions without coordinates: prob_rate_rwr_max, swath_width, time, yr_day_utc
Data variables:
    rain_stats  (prob_rate_rwr_max, swath_width, time) float32 ...
    latitude    (swath_width, time) float32 ...
    longitude   (swath_width, time) float32 ...
    time_vars   (yr_day_utc, time) float32 ...

In [17]:
[mpl.cm.get_cmap('jet', 6)(i) for i in np.linspace(0,1,6)]

[(0.0, 0.0, 0.5, 1.0),
 (0.0, 0.3, 1.0, 1.0),
 (0.16129032258064513, 1.0, 0.8064516129032259, 1.0),
 (0.806451612903226, 1.0, 0.1612903225806449, 1.0),
 (1.0, 0.40740740740740733, 0.0, 1.0),
 (0.5, 0.0, 0.0, 1.0)]

In [14]:
    colors = [mpl.cm.get_cmap('viridis')(i) for i in np.linspace(0,1,6)]


In [19]:
colors

[(0.267004, 0.004874, 0.329415, 1.0),
 (0.253935, 0.265254, 0.529983, 1.0),
 (0.163625, 0.471133, 0.558148, 1.0),
 (0.134692, 0.658636, 0.517649, 1.0),
 (0.477504, 0.821444, 0.318195, 1.0),
 (0.993248, 0.906157, 0.143936, 1.0)]

In [ ]:
def plot_granule(plot_gran, latlon_range=None):

    df_gran = class_by_gran.get_group(plot_gran)
    date = df_gran.datetime.iloc[0]

    modis_data, myd03_data = load_MODIS_granule(plot_gran)
    ascat_data = get_divergence(date)
    MERRA_data = get_MERRA(date)
    MERRA_SEP_data = get_SEP_MERRA(date)
    amsr_data = get_AMSR(date)
    precip_data = get_precip(date)

    fig, [[ax0, ax1, ax2], [ax3, ax4, ax5]] = plt.subplots(figsize=(12,12), ncols=3, nrows=2, sharex=True, sharey=True)#, constrained_layout=True)

    fig.subplots_adjust(wspace=0.05, hspace=0.05)
    mn = []
    class_cmap =  mpl.cm.get_cmap('viridis', 6)
    colors = [mpl.cm.get_cmap('viridis')(i) for i in np.linspace(0,1,6)]


    lats = myd03_data.Latitude.values
    lons = myd03_data.Longitude.values%360
    zenith = myd03_data.SensorZenith.values
    good_rows = np.nonzero(np.all(zenith<45, axis=0))
    lats_trim = lats[:,slice(np.min(good_rows), np.max(good_rows))]
    lons_trim = lons[:,slice(np.min(good_rows), np.max(good_rows))]
    lat_0 = np.mean(lats)
    lat_range = np.array([np.min(lats_trim), np.max(lats_trim)])
    lon_0 = np.mean(lons)
    lon_range = np.array([np.min(lons_trim), np.max(lons_trim)])

    if latlon_range:
        lat_range = latlon_range['lat']
        lon_range = latlon_range['lon']

    for ax in fig.axes:
        map_kwargs = dict(projection='cyl', resolution='l',
                          llcrnrlat=lat_range[0], urcrnrlat=lat_range[1],
                          llcrnrlon=lon_range[0], urcrnrlon=lon_range[1],
                          lat_0=lat_0, lon_0=lon_0)
        m = Basemap(**map_kwargs, ax=ax)
        mn.append(m)
        parallels = np.arange(-90.,90,5.)
        par_labs = [1, 0, 0, 0] if ax in [ax0, ax3] else [0, 0, 0, 0]
        mer_labs = [0, 0, 0, 1] if ax in [ax3, ax4, ax5] else [0, 0, 0, 0]
        m.drawparallels(parallels,labels=par_labs)
        meridians = np.arange(0.,351.,5.)
        m.drawmeridians(meridians,labels=mer_labs)
        m.drawcoastlines()
        m.drawcountries()

        cat_color = [utils.ordering.index(i) for i in df_gran.cat]

#         if ax == ax0:
#             m.scatter(lons[112::128,:], lats[112::128,:], c='w', s=0.3, latlon=True, zorder=8)
#             m.scatter(lons[:,64::128], lats[:,64::128], c='w', s=0.3, latlon=True, zorder=8)
        sc = m.scatter(df_gran.lon%360, df_gran.lat, c=cat_color, cmap=class_cmap, s=40, latlon=True, vmin=-0.5, vmax=5.5,
                      zorder=10, edgecolors='k', alpha=0.8)

    #plot 0: Reflectance with classifications
    rgb = get_rgb_reflectance(modis_data)
    mesh_rgb = rgb[:, :-1, :] #this also does magic
    colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
    colorTuple = np.insert(colorTuple,3,1.0,axis=1) #this does magic
    mn[0].pcolormesh(lons, lats, rgb[:,:,1], latlon=True,color=colorTuple)
    ax0c, _ = make_axes(ax0, location='top', pad=0.1, shrink=0.9)
    ax0c.axis('off')
    legend_elements = [mpl.lines.Line2D([0], [0], marker='o', linestyle="None", mfc=colors[i], mec='w', label=utils.short_labels[c], alpha=0.8) 
                       for i,c in enumerate(utils.ordering)]
    ax0c.legend(handles=legend_elements, loc='lower left', bbox_to_anchor= (0.0, 0.0), fontsize=11, ncol=2, 
               borderaxespad=0, frameon=True, handletextpad=0.3, columnspacing=0.5, facecolor='gainsboro')

    # plot 1: ASCAT divergence
    ax1c, _ = make_axes(ax1, location='top', pad=0.1, shrink=0.9)
    ax1.set_facecolor('gainsboro')
    pc = mn[1].pcolormesh(ascat_data.longitude%360, ascat_data.latitude, ascat_data.div.sel(orbit_segment=0)*1e6,
                    vmin=-40, vmax=40, cmap='bwr')
    cb = fig.colorbar(pc, cax=ax1c, orientation='horizontal')
    cb.set_label('ASCAT sfc. div. ($10^{-6}$ s$^{-1}$)')
    ax1c.xaxis.tick_top()
    ax1c.xaxis.set_label_position('top') 

    #plot 2: MERRA surface divergence
    ax2c, _ = make_axes(ax2, location='top', pad=0.1, shrink=0.9)
    pc = mn[2].pcolormesh(MERRA_data.lon%360, MERRA_data.lat, MERRA_data.sfc_div*1e6,
                    vmin=-40, vmax=40, cmap='bwr')
    pc2 = mn[2].pcolormesh(MERRA_SEP_data.lon%360, MERRA_SEP_data.lat, MERRA_SEP_data.sfc_div*1e6,
                    vmin=-40, vmax=40, cmap='bwr')
    cb = plt.colorbar(pc, cax=ax2c, orientation='horizontal')
    cb.set_label('MERRA sfc. div ($10^{-6}$ s$^{-1}$)')
    ax2c.xaxis.tick_top()
    ax2c.xaxis.set_label_position('top') 

    #plot 3: AMSR-based precip
    ax3c, _ = make_axes(ax3, location='bottom', pad=0.1, shrink=0.9)
    ax3.set_facecolor('gainsboro')
    x = precip_data.longitude.values%360
    y = precip_data.latitude.values
    z= precip_data.rain_stats.isel(prob_rate_rwr_max=1).values
    z[np.isnan(x)] = np.nan
    z[np.isnan(y)] = np.nan
    x[np.isnan(x)] = 1
    y[np.isnan(y)] = 1
    pc = mn[3].pcolormesh(x, y, z, cmap='Reds', vmin=0, vmax=4)
    cb = fig.colorbar(pc, cax=ax3c, orientation='horizontal')
    cb.set_label('AMSR precip rate (mm hr$^{-1}$)')

    #plot 4: AMSR CWV
    ax4c, _ = make_axes(ax4, location='bottom', pad=0.1, shrink=0.9)
    ax4.set_facecolor('gainsboro')
    pc = mn[4].pcolormesh(amsr_data.longitude%360, amsr_data.latitude, amsr_data, cmap='Blues')
    cb = plt.colorbar(pc, cax=ax4c, orientation='horizontal')
    cb.set_label('AMSR CWV (kg m$^{-2}$)')

    #plot 5: MERRA 700mb divergence
    ax5c, _ = make_axes(ax5, location='bottom', pad=0.1, shrink=0.9)
    MERRA_subs_700 = -MERRA_data.dzdt.sel(lev=700)/MERRA_data.H.sel(lev=700)
    MERRA_SEP_subs_700 = -MERRA_SEP_data.dzdt.sel(lev=700)/MERRA_SEP_data.H.sel(lev=700)
    pc = mn[5].pcolormesh(MERRA_data.lon%360, MERRA_data.lat, MERRA_subs_700*1e6,
                    vmin=-10, vmax=10, cmap='bwr')
    mn[5].pcolormesh(MERRA_SEP_data.lon%360, MERRA_SEP_data.lat, MERRA_SEP_subs_700*1e6,
                    vmin=-10, vmax=10, cmap='bwr')
    cb = plt.colorbar(pc, cax=ax5c, orientation='horizontal')
    cb.set_label('MERRA large-scale div ($10^{-6}$ s$^{-1}$)')
#     

    for i, axi in enumerate(fig.axes):
        if i > 5:
            break
        letter = chr(ord('a') + i)
        axi.text(0.01, 0.99, f'({letter})', fontsize=12, horizontalalignment='left', verticalalignment='top',
                 transform=axi.transAxes, zorder=11, backgroundcolor='w')


    
#     fig.savefig(f'/home/disk/p/jkcm/plots/measures/big_granules/{plot_gran}.png', bbox_inches='tight')
    return fig
for plot_gran in [best_gran]:
    fig = plot_granule(plot_gran, latlon_range={'lat': (13, 22), 'lon': np.array([-146, -134])%360})
    fig.set_size_inches(12, 7)
#     fig.suptitle(plot_gran, y=1.02)
    
    fig.savefig(f'/home/disk/p/jkcm/plots/measures/final/{plot_gran}_subset.png', bbox_inches='tight')


In [ ]:
print(date)
print(MERRA_data.time.values)
print(ascat_data.time.values)
print(amsr_data.time.values)

In [ ]:
include_divergence = True

# for gran in nice_grans[0:1]:
for gran in [hurricane_gran]:
    df_gran = class_by_gran.get_group(gran)
    modis_data, myd03_data = load_MODIS_granule(gran)
    rgb = get_rgb_reflectance(modis_data)
    lats = myd03_data.Latitude.values
    lons = myd03_data.Longitude.values%360
    zenith = myd03_data.SensorZenith.values

    good_rows = np.nonzero(np.all(zenith<45, axis=0))
    lats_trim = lats[:,slice(np.min(good_rows), np.max(good_rows))]
    lons_trim = lons[:,slice(np.min(good_rows), np.max(good_rows))]
    
    if include_divergence:
        fig, axn = plt.subplots(figsize=(20, 12), ncols=2, sharex=True, sharey=True)
        ax1 = axn[0]
    else:
        fig, ax1 = plt.subplots(figsize=(10,10))
        axn = [ax1]
    
    lat_0 = np.mean(lats)
    lat_range = np.array([np.min(lats_trim), np.max(lats_trim)])
    lon_0 = np.mean(lons)
    lon_range = np.array([np.min(lons_trim), np.max(lons_trim)])

    mn = []
    cmap = mpl.cm.get_cmap('gist_ncar', 6)
    for ax in axn:
        map_kwargs = dict(projection='cyl', resolution='l',
                          llcrnrlat=lat_range[0], urcrnrlat=lat_range[1],
                          llcrnrlon=lon_range[0], urcrnrlon=lon_range[1],
                          lat_0=lat_0, lon_0=lon_0)
        m = Basemap(**map_kwargs, ax=ax)
        mn.append(m)
        parallels = np.arange(-90.,90,5.)
        m.drawparallels(parallels,labels=[False,True,True,False])
        meridians = np.arange(0.,351.,5.)
        m.drawmeridians(meridians,labels=[True,False,False,True])
        m.drawcoastlines()
        m.drawcountries()

        m.scatter(lons[112::128,:], lats[112::128,:], c='w', s=2, latlon=True, zorder=8)
        m.scatter(lons[:,64::128], lats[:,64::128], c='w', s=2, latlon=True, zorder=8)
        sc = m.scatter(df_gran.lon%360, df_gran.lat, c=df_gran.cat, cmap=cmap, s=80, latlon=True, vmin=-0.5, vmax=5.5,
                      zorder=10)
        
        
    mesh_rgb = rgb[:, :-1, :] #this also does magic
    colorTuple = mesh_rgb.reshape((mesh_rgb.shape[0] * mesh_rgb.shape[1]), 3)
    colorTuple = np.insert(colorTuple,3,1.0,axis=1) #this does magic
    mn[0].pcolormesh(lons, lats, rgb[:,:,1], latlon=True,color=colorTuple)
    cb = plt.colorbar(sc, ax=ax1, orientation='horizontal')
    cb.set_ticks(sorted(utils.short_labels))
    cb.set_ticklabels([utils.short_labels[i] for i in cb.get_ticks()])
    cb.ax.set_xticklabels(cb.ax.get_xticklabels(), rotation=45)
    ax1.set_title(gran)
    
    if include_divergence:
        axn[1].set_facecolor('gainsboro')
        date =  df_gran.datetime.iloc[0]
        ascat_data = get_divergence(date)
        pc = mn[1].pcolormesh(ascat_data.longitude%360, ascat_data.latitude, ascat_data.div.sel(orbit_segment=0)*1e5,
                        vmin=-4, vmax=4, cmap='bwr')
#         pc = mn[1].pcolormesh(ascat_data.longitude%360, ascat_data.latitude, ascat_data.windspd.sel(orbit_segment=1)*1e5,
#                         vmin=0, vmax=15, cmap='reds')
        cb = plt.colorbar(pc, ax=axn[1], orientation='horizontal')
        cb.set_label('ASCAT divergence ($10^{-5}$ s$^{-1}$)')
        axn[1].set_title(f'ASCAT morning div: {str(ascat_data.time.values)[:10]}')
     
#     fig.savefig(f'/home/disk/p/jkcm/plots/measures/variable_granules/with_div/{gran}_morn.jpg', dpi=100)
#     plt.close()


In [ ]:
    i=0
    lat, lon, time = cset_class_df.lat.iloc[i], cset_class_df.lon.iloc[i], cset_class_df.datetime.iloc[i]
    date=time
    date_adj = time-np.timedelta64(5, 'h')
    scat_data = xr.open_dataset(r'/home/disk/eos9/jkcm/Data/ascat/rss/all/ascat_unified_'+f'{date_adj.year}-{date_adj.month:02}.nc')
    scat_data = scat_data.sel(time=(time-np.timedelta64(5, 'h')).date(), method='nearest', orbit_segment=0)
    scat_data = utils.get_ascat_divergence(scat_data)
    
    
    xy = scat_data.sel(latitude=slice(lat-0.5, lat+0.5), longitude=slice(lon-0.5, lon+0.5))
    if not xy.windspd.ndim == 2:
        raise ValueError('hey boss I thought this was gonna only be 2-dimensional.')
#     np.nanmean(xy.div.values), np.nanstd(xy.div.values), np.nanmean((xy.time.values-time)/np.timedelta64(1, 'h'))

((xy.time.values+(xy.mingmt.values*60*1e3).astype('timedelta64[ms]'))-time.to_datetime64())/np.timedelta64(1, 'h')

In [ ]:
import datetime as dt
date = dt.datetime(2015,7,15)

In [ ]:
def get_MERRA_data(date):
    MERRA_file = f'/home/disk/eos4/jkcm/Data/CSET/MERRA/sfc/MERRA2_400.inst1_2d_asm_Nx.{date.strftime("%Y%m%d")}.nc4.nc'
    MERRA_data = xr.open_dataset(MERRA_file)
    MERRA_data['lon'] = MERRA_data.lon%360
    MERRA_data['div'] = (('time', 'lat', 'lon'), 
                         utils.get_div_from_u_v(MERRA_data.U10M, MERRA_data.V10M, MERRA_data.lat.values, MERRA_data.lon.values))
    return MERRA_data

In [ ]:
MERRA_data.div

In [ ]:

for date in [date]:
    ascat_data = get_divergence(date)
    MERRA_data = get_MERRA_data(date)
    MERRA_data = MERRA_data.sel(time=
    lats = MERRA_data.lat
    lons = MERRA_data.lon
    lat_0 = np.mean(lats)
    lat_range = np.array([np.min(lats_trim), np.max(lats_trim)])
    lon_0 = np.mean(lons)
    lon_range = np.array([np.min(lons_trim), np.max(lons_trim)])

    mn = []
    fig, axn = plt.subplots(figsize=(20, 12), ncols=2, sharex=True, sharey=True)
    for ax in axn:
        map_kwargs = dict(projection='cyl', resolution='l',
                          llcrnrlat=lat_range[0], urcrnrlat=lat_range[1],
                          llcrnrlon=lon_range[0], urcrnrlon=lon_range[1],
                          lat_0=lat_0, lon_0=lon_0)
        m = Basemap(**map_kwargs, ax=ax)
        mn.append(m)
        parallels = np.arange(-90.,90,5.)
        m.drawparallels(parallels,labels=[False,True,True,False])
        meridians = np.arange(0.,351.,5.)
        m.drawmeridians(meridians,labels=[True,False,False,True])
        m.drawcoastlines()
        m.drawcountries()
    
    axn[1].set_facecolor('gainsboro')
    pc = mn[1].pcolormesh(ascat_data.longitude%360, ascat_data.latitude, ascat_data.div.sel(orbit_segment=0)*1e5,
                    vmin=-4, vmax=4, cmap='bwr')
    cb = plt.colorbar(pc, ax=axn[1], orientation='horizontal')
    cb.set_label('ASCAT divergence ($10^{-5}$ s$^{-1}$)')
    axn[1].set_title(f'ASCAT morning div: {str(ascat_data.time.values)[:10]}')

    axn[0].set_facecolor('gainsboro')
    pc = mn[0].pcolormesh(MERRA_data.lon%360, MERRA_data.lat, ascat_data.div.sel(orbit_segment=0)*1e5,
                    vmin=-4, vmax=4, cmap='bwr')
    cb = plt.colorbar(pc, ax=axn[1], orientation='horizontal')
    cb.set_label('ASCAT divergence ($10^{-5}$ s$^{-1}$)')
    axn[1].set_title(f'ASCAT morning div: {str(ascat_data.time.values)[:10]}')

    fig.savefig(f'/home/disk/p/jkcm/plots/measures/variable_granules/with_div/{gran}_MERRA_ASCAT.jpg', dpi=100)
    plt.close()


In [ ]:
import glob

In [ ]:
sfcs = glob.glob(r'/home/disk/eos4/jkcm/Data/MERRA/sfc/*.nc4')
pres = glob.glob(r'/home/disk/eos4/jkcm/Data/MERRA/pressure/*.nc')

In [ ]:
p1 = [i[-15:-7] for i in pres]
s1 = [i[-16:-8] for i in sfcs]
missing_files = [i for i in s1 if i not in p1]

In [ ]:
textfile = glob.glob(r'/home/disk/eos4/jkcm/Data/MERRA/pressure/*.txt')[0]
newfile = textfile[:-4]+'_updated.txt'

In [ ]:
with open(textfile, 'rb') as infile, open(newfile, 'wb') as outfile:
    mc=0
    lines = infile.readlines()
    for l in lines[1:]:
        f = str(l).split('SHORTNAME')[0].split('LABEL')[1]
        if f[-16:-8] in missing_files:
            outfile.write(l)

In [ ]:
textfile